In [5]:
# %pip install wikipedia
# %pip install numexpr

## Agent Example

In [6]:
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
tools = load_tools(['wikipedia', "llm-math"], llm=llm)

agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    return_intermediate_steps=True,
    verbose=True,
)

C:\Users\Owner\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [7]:
from langchain.callbacks import StdOutCallbackHandler

question = """
What is the current American president's age? 
Return the age multiplied by 82
"""

handler = StdOutCallbackHandler()
response = agent(
    {"input": question}, 
    callbacks=[handler]
)

response

C:\Users\Owner\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Calculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [wikipedia, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Though

{'input': "\nWhat is the current American president's age? \nReturn the age multiplied by 82\n",
 'output': '6642',
 'intermediate_steps': [(AgentAction(tool='wikipedia', tool_input='Joe Biden', log='I need to find the current American president\'s age first, then multiply it by 82.\nAction: wikipedia\nAction Input: "Joe Biden"'),
   "Page: Joe Biden\nSummary: Joseph Robinette Biden Jr. (born November 20, 1942) is an American politician serving as the 46th and current president of the United States since 2021. A member of the Democratic Party, he served as the 47th vice president from 2009 to 2017 under President Barack Obama and represented Delaware in the U.S. Senate from 1973 to 2009.\nBorn in Scranton, Pennsylvania, Biden moved with his family to Delaware in 1953. He graduated from the University of Delaware in 1965 and from Syracuse University in 1968. He was elected to the New Castle County Council in 1970 and the U.S. Senate in 1972. As a senator, Biden drafted and led the effor

In [8]:
response['intermediate_steps'][0][0]

AgentAction(tool='wikipedia', tool_input='Joe Biden', log='I need to find the current American president\'s age first, then multiply it by 82.\nAction: wikipedia\nAction Input: "Joe Biden"')

In [9]:
import wikipedia

wikipedia.search('current American president')

['President of the United States',
 'List of presidents of the United States',
 'List of current vice presidents and designated acting presidents',
 'List of current presidents of legislatures',
 'Vice President of the United States',
 'Historical rankings of presidents of the United States',
 'George Washington',
 'Abraham Lincoln',
 'List of current United States senators',
 'List of presidents of the United States by age']

In [10]:
wikipedia.summary('President of the United States')

'The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.\nThe power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansions during the presidencies of Franklin D. Roosevelt and George W. Bush. In modern times, the president is one of the world\'s most powerful political figures and the leader of the world\'s only remaining superpower. As the leader of the nation with the largest economy by nominal GDP, the president possesses significant domestic and international hard and soft power. For much of the

In [11]:
wikipedia.summary('List of presidents of the United States')

'The president of the United States is the head of state and head of government of the United States, indirectly elected to a four-year term via the Electoral College. The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. Since the office was established in 1789, 45 men have served in 46 presidencies. The first president, George Washington, won a unanimous vote of the Electoral College. Grover Cleveland served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to the discrepancy between the number of presidencies and the number of individuals who have served as president.\nThe presidency of William Henry Harrison, who died 31 days after taking office in 1841, was the shortest in American history. Franklin D. Roosevelt served the longest, over twelve years, before dying early in his fourth term in 1945. He is the only U.S. president to have se

In [13]:
from langchain import LLMMathChain

chain = LLMMathChain.from_llm(llm, verbose=True)
chain.run('45 * 89', callbacks=[handler])

C:\Users\Owner\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMMathChain chain...
45 * 89

> Entering new LLMChain chain...
Prompt after formatting:
Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${Question with math problem.}
```text
${single line mathematical expression that solves the problem}
```
...numexpr.evaluate(text)...
```output
${Output of running the code}
```
Answer: ${Answer}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: 45 * 89


> Finished chain.
```text
45 * 89
```
...numexpr.evaluate("45 * 89")...

Answer: 4005
> Finished chain.


'Answer: 4005'

## Custom Tools

In [ ]:
from langchain.document_loaders import PyPDFLoader

file_path = 'mixed_data/element_of_SL.pdf'

loader = PyPDFLoader(file_path=file_path)
data = loader.load_and_split()

In [15]:
data[:5]

[Document(metadata={'source': 'mixed_data/element_of_SL.pdf', 'page': 0}, page_content='Springer Series in Statistics\nTrevor Hastie\nRobert TibshiraniJerome FriedmanSpringer Series in Statistics\nThe Elements of\nStatistical Learning\nData Mining, Inference, and Prediction\nThe Elements of Statistical LearningDuring the past decade there has been an explosion in computation and information tech-\nnology. With it have come vast amounts of data in a variety of fields such as medicine, biolo-gy, finance, and marketing. The challenge of understanding these data has led to the devel-opment of new tools in the field of statistics, and spawned new areas such as data mining,machine learning, and bioinformatics. Many of these tools have common underpinnings butare often expressed with different terminology. This book describes the important ideas inthese areas in a common conceptual framework. While the approach is statistical, theemphasis is on concepts rather than mathematics. Many examples 

In [16]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(
    data, 
    embeddings, 
    collection_name="statistical_learning"
)

C:\Users\Owner\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [17]:
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever()
)

In [18]:
chain.run('What is Machine Learning?', callbacks=[handler])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
This is page 1
Printer: Opaque this
1
Introduction
Statistical learning plays a key role in many areas of science, ﬁnance and
industry. Here are some examples of learning problems:
•Predict whether a patient, hospitalized due to a heart attack, will
have a second heart attack. The prediction is to be based on demo-
graphic, diet and clinical measurements for that patient.
•Predict the price of a stock in 6 months from now, on the basis of
company performance measures and economic data.
•Identify the numbers in a handwritten ZIP code, from a digitized
image.
•Estimate the amount of glucose in the blood of a diabetic person,
from the infrared absorption spectrum of

'Machine learning is a field of study that involves the development of algorithms and statistical models that enable computers to learn and improve from experience without being explicitly programmed. It is a subset of artificial intelligence that focuses on the creation of systems that can automatically learn and improve from data.'

In [19]:
from langchain.agents import Tool

description = """
useful for when you need to answer questions 
about Machine Learning and Statistical Learning. 
Input should be a fully formed question.
"""

retrieval_tool = Tool(
    name="Statistical Learning Book",
    func=chain.run,
    description=description,
)

In [20]:
tools = load_tools(['wikipedia', "llm-math"], llm=llm)

tools.append(retrieval_tool)

agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    return_intermediate_steps=True,
)

In [21]:
question = """
Can we use machine learning to predict the age of the president?
"""

response = agent(
    {"input": question}, 
    callbacks=[handler]
)

response



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Calculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.
Statistical Learning Book(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - 
u

{'input': '\nCan we use machine learning to predict the age of the president?\n',
 'output': 'Yes, machine learning can be used to predict the age of individuals.',
 'intermediate_steps': [(AgentAction(tool='Statistical Learning Book', tool_input='Can machine learning be used to predict the age of individuals?', log='Since this question is related to Machine Learning, we should use the Statistical Learning Book tool to get a better understanding of how machine learning can be applied in predicting age.\nAction: Statistical Learning Book\nAction Input: Can machine learning be used to predict the age of individuals?'),
   'Yes, machine learning can be used to predict the age of individuals. In the field of statistics and data mining, predictive modeling techniques can be applied to infer the age of individuals based on various features and data points available in a dataset. It is a common application of machine learning algorithms to predict demographic information such as age.')]}